In [95]:
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import json
import glob
import os
from PIL import Image
import random

import bottleneck
from resnet_pytorch import ResNet

## Pre-processing data

### Labels

In [96]:
# loading labels
files = glob.glob('../../1_data_collection/.data/*.json')
number_of_files = 20

labels = []  # Initialize a list to store data from all files

if not files:
    print("No files found. Check the directory path.")
else:
    for file_path in files[:number_of_files]:
        try:
            with open(file_path, 'r') as file:
                # Load the content of the current file
                file_data = json.load(file)
                if "coordinates" in file_data:
                    labels.append(file_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {file_path}: {e}")
        except Exception as e:
            print(f"Error opening or reading {file_path}: {e}")

# Output how many files were successfully loaded
print(f"Loaded data from {len(labels)} files.")


Loaded data from 19 files.


In [97]:
# Just load the first image, avoid Multiplayer
labels[0]["coordinates"]

[25.727112, 30.561267]

### Input images

In [98]:
# loading images
files = glob.glob('../../1_data_collection/.data/*.png')

if not files:
    print("No files found. Check the directory path.")
else:
    images = [] 
    for file_path in files[:number_of_files]:
        try:
            # channels, height, width is the pytorch convention
            with Image.open(file_path) as img:
                img_array = np.array(img)
                img_array = img_array.transpose((2, 0, 1))
                #images.append(img_array)
                images.append(file_path)
        except IOError as e:
            print(f"Error opening or reading {file_path}: {e}")

# Output how many files were successfully processed
print(f"Processed {len(images)} image files.")
#display(Image.open(images[0]))

Processed 20 image files.


In [99]:
# Inside the loop, convert the image to a numpy array and store it
with Image.open(file_path) as img:
    width, height = img.size
    img_array = np.array(img)
    print(f"Image size (height, width): {height}x{width}")
    print(f"Image array shape: {img_array.shape}")
    print(f"Image mode: {img.mode}")
    print(f"Image format: {img.format}")

Image size (height, width): 180x320
Image array shape: (180, 320, 4)
Image mode: RGBA
Image format: PNG


In [100]:
#images[0].shape
images[0]

'../../1_data_collection/.data/geoguessr_location_singleplayer_rm8goISVT6SLjGR9_4_resized.png'

In [112]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [113]:
# split before to train, validation, and test
dataset = CustomImageDataset(images, labels, transform=transform)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [118]:
loader

In [116]:
loader.dataset[0][0].shape

torch.Size([3, 100, 100])

In [120]:
# Assuming loader is already defined as you've shown
for batch in loader:
    print(batch)
    #images, labels = batch
    print(f"Batch size: {len(images)}")  # This will print 2 or the final batch size if it's different
    print(f"First image in batch shape: {images[0].shape}")  # Shape of the first image tensor in the batch
    print(f"First label in batch: {labels[0]}")  # First label in the batch
    
    # If you only want to see one batch, you can break the loop
    break


TypeError: list indices must be integers or slices, not list

### Splitting dataset into training, validation and testing sets

In [43]:
# create dataset
data = {}
data["input"] = labels

# Define split ratios
train_ratio = 0.7
val_ratio = 0.20
test_ratio = 0.10

# Shuffle data randomly
random.shuffle(data["input"])

# Calculate split indices
total_images = len(data["input"])
train_end = int(train_ratio * total_images)
val_end = train_end + int(val_ratio * total_images)

# Split the data
train_data = data["input"][:train_end]
val_data = data["input"][train_end:val_end]
test_data = data["input"][val_end:]

print(f"Train set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


Train set size: 14
Validation set size: 4
Test set size: 2


In [44]:
#Loading dataet and crearting, split before!!!
dataset = DatasetDict({
    "train": train_data,
    "validation": val_data,
    "test": test_data
})

In [45]:
dataset

DatasetDict({
    train: [{'coordinates': [9.940982, 33.602745], 'duration': 10.401}, {'coordinates': [10.23696, -28.960029], 'duration': 10.434}, {'coordinates': [17.895419, -11.1704], 'duration': 9.982}, {'coordinates': [28.444196, 30.43578], 'duration': 10.419}, {'coordinates': [30.621082, 22.407837], 'duration': 9.81}, {'coordinates': [25.727112, 30.561267], 'duration': 10.427}, {'coordinates': [47.887935, -27.093841], 'duration': 10.309}, {'coordinates': [48.02805, 32.543271], 'duration': 10.303}, {'country': 'Tunisia', 'guesses': {'1': {'incorrect': []}, '2': {'incorrect': []}, '3': {'incorrect': []}, '4': {'incorrect': []}, '5': {'incorrect': []}, '6': {'incorrect': []}, '7': {'incorrect': []}, '8': {'incorrect': []}, '9': {'incorrect': []}, '10': {'incorrect': []}, '11': {'incorrect': []}, '12': {'incorrect': ['Türkiye', 'United States']}, '13': {'incorrect': ['Türkiye', 'United States']}, '14': {'incorrect': ['Türkiye', 'United States']}, '15': {'incorrect': ['Türkiye', 'Unite

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

### Creating Dataloaders

In [ ]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

## Model

In [ ]:
model = ResNet.from_pretrained('resnet18', num_classes=2)

Loaded pretrained weights for resnet18.


In [ ]:
#print(model)

## Training

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
net = ResNet50(10).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [ ]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')